In [4]:
# Requrements
%pip install requests-html
%pip install --upgrade lxml_html_clean
%pip install selenium


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# Get all links containing countries bank-code.net
from selenium import webdriver
from selenium.webdriver.common.by import By

browser = webdriver.Chrome()

browser.get("https://bank-code.net/country/starts-with-letter-A")
# get list in <ul class="nav nav-pills"></ul>
ul = browser.find_element(By.CLASS_NAME, "nav-pills")
# get list items in <li></li>
lis = ul.find_elements(By.TAG_NAME, "li")
# get links in <a></a>
links = [li.find_element(By.TAG_NAME, "a") for li in lis]
# get href attribute of <a></a>
hrefs = [link.get_attribute("href") for link in links]
# print hrefs
print(hrefs)

['https://bank-code.net/country/starts-with-letter-A', 'https://bank-code.net/country/starts-with-letter-B', 'https://bank-code.net/country/starts-with-letter-C', 'https://bank-code.net/country/starts-with-letter-D', 'https://bank-code.net/country/starts-with-letter-E', 'https://bank-code.net/country/starts-with-letter-F', 'https://bank-code.net/country/starts-with-letter-G', 'https://bank-code.net/country/starts-with-letter-H', 'https://bank-code.net/country/starts-with-letter-I', 'https://bank-code.net/country/starts-with-letter-J', 'https://bank-code.net/country/starts-with-letter-K', 'https://bank-code.net/country/starts-with-letter-L', 'https://bank-code.net/country/starts-with-letter-M', 'https://bank-code.net/country/starts-with-letter-N', 'https://bank-code.net/country/starts-with-letter-O', 'https://bank-code.net/country/starts-with-letter-P', 'https://bank-code.net/country/starts-with-letter-Q', 'https://bank-code.net/country/starts-with-letter-R', 'https://bank-code.net/coun

In [6]:
# for each href in hrefs get list of countries in <div class="span4Style"></div>
country_list_hrefs = []
for href in hrefs:
    browser.get(href)
    divs = browser.find_elements(By.CLASS_NAME, "span4Style")
    # get second <a></a> for each <div></div>
    for div in divs:
        as_ = div.find_elements(By.TAG_NAME, "a")[1]
        country_list_hrefs.append(as_.get_attribute("href"))
print(country_list_hrefs)

['https://bank-code.net/country/AFGHANISTAN-%28AF%29.html', 'https://bank-code.net/country/ALBANIA-%28AL%29.html', 'https://bank-code.net/country/ALGERIA-%28DZ%29.html', 'https://bank-code.net/country/AMERICAN-SAMOA-%28AS%29.html', 'https://bank-code.net/country/ANDORRA-%28AD%29.html', 'https://bank-code.net/country/ANGOLA-%28AO%29.html', 'https://bank-code.net/country/ANGUILLA-%28AI%29.html', 'https://bank-code.net/country/ANTIGUA-AND-BARBUDA-%28AG%29.html', 'https://bank-code.net/country/ARGENTINA-%28AR%29.html', 'https://bank-code.net/country/ARMENIA-%28AM%29.html', 'https://bank-code.net/country/ARUBA-%28AW%29.html', 'https://bank-code.net/country/AUSTRALIA-%28AU%29.html', 'https://bank-code.net/country/AUSTRIA-%28AT%29.html', 'https://bank-code.net/country/AZERBAIJAN-%28AZ%29.html', 'https://bank-code.net/country/BAHAMAS-%28BS%29.html', 'https://bank-code.net/country/BAHRAIN-%28BH%29.html', 'https://bank-code.net/country/BANGLADESH-%28BD%29.html', 'https://bank-code.net/country/BA

In [7]:
# save country list hrefs to file so we can use it later
with open("country_list_hrefs.txt", "w") as file:
    for href in country_list_hrefs:
        file.write(href + "\n")

In [8]:
# get bank table from each page in #tableID
# Get rows from table into object list
# row contains 5 columns, No, Bank Name, Branch, City , Swift Code
def getBankDetails(href):
    browser.get(href)
    table = browser.find_element(By.ID, "tableID").find_element(By.TAG_NAME, "tbody")
    rows = table.find_elements(By.TAG_NAME, "tr")
    # print row data
    bank_details = []
    for row in rows:
        data = row.find_elements(By.TAG_NAME, "td")
        # if data is not empty
        if len(data) == 5:
            bank_details.append({
                "BankName": data[1].text,
                "Branch": data[2].text,
                "City": data[3].text,
                "SwiftCode": data[4].text
            })
        else:
            for d in data:
                print(d.text)
    return bank_details

In [9]:
# Check if page is paginated
# for each country list href in country_list_hrefs check if element <nav aria-label="Page navigation"></nav> exists
#  if exists create list of hrefs 
# where if country_list_hrefs = https://bank-code.net/country/KENYA-%28KE%29.html
# then hrefs = [ https://bank-code.net/country/KENYA-%28KE%29.html, https://bank-code.net/country/KENYA-%28KE%29/50.html, https://bank-code.net/country/KENYA-%28KE%29/100.html ... ]
# else hrefs = [ https://bank-code.net/country/KENYA-%28KE%29.html ]

def getBankTableList(country_list_href):
    browser.get(country_list_href)
    # check if pagination exists
    try:
        nav = browser.find_element(By.CLASS_NAME, "pagination")
    except:
        nav = None
    hrefs = [country_list_href]
    bank_details = []
    if nav:
        # generate links for 50 ... 100 ... 150 .... 1000
        for i in range(1, 41):
            # remove .html from country_list_href
            country_list_href = country_list_href.replace(".html", "")
            url = f"{country_list_href}/{i * 50}.html"
            hrefs.append(url)
        for href in hrefs:
            bank_details.extend(getTableDataIfExists(href))
    # return
    print("Pages: ", bank_details)
    return bank_details

# check if url exists
def getTableDataIfExists(url):
    browser.get(url)
    # url withouth .html matches current_url with .html return True else False
    url_ = url
    exists =  url.replace(".html", "") == browser.current_url.replace(".html", "")
    if exists:
        return getBankDetails(url_)
    return []


In [10]:

import csv

# check if file exists
def checkIfFileExists(file):
    try:
        with open(file, "r") as f:
            return True
    except:
        return False

# create checked.txt file if not exists
def createCheckedTxt():
    if not checkIfFileExists("checked.txt"):
        with open("checked.txt", "w") as f:
            f.write("")

# append href to checked.txt file
def appendToChecked(href):
    with open("checked.txt", "a", encoding="utf-8") as f:
        f.write(href + "\n")

# read checked.txt file
def readChecked():
    with open("checked.txt", "r") as f:
        return f.read().split("\n")

# create bank details csv file if not exists
def createBankDetailsCsv():
    if not checkIfFileExists("bank_details.csv"):
        with open("bank_details.csv", "w") as f:
            writer = csv.DictWriter(f, fieldnames=["BankName", "Branch", "City", "SwiftCode"])
            writer.writeheader()

# append bank details to csv file
def appendToBankDetailsList(bank_details):
    print("Appending ", bank_details)
    with open("bank_details.csv", "a", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=["BankName", "Branch", "City", "SwiftCode"])
        for bank_detail in bank_details:
            writer.writerow(bank_detail)

# get country list hrefs
def getCountryListHrefs():
    with open("country_list_hrefs.txt", "r") as f:
        return f.read().split("\n")

createCheckedTxt()
createBankDetailsCsv()

In [11]:
# Note 1: incase this script is stopped you can start from here granted all the above code has been run
# Note 2: if you want to start from the beginning delete checked.txt, country_list_hrefs.txt and bank_details.csv
# Note 3: This script does not take into account rate limiting, you can add a sleep function to slow down the requests
# Get bank details for each href in country_list_hrefs
# When href is checked add to checked list txt file
# Save bank details to csv
checked = readChecked()
country_list_hrefs = getCountryListHrefs()
for country_list_href in country_list_hrefs:
    if country_list_href in checked:
        continue
    appendToBankDetailsList(getBankTableList(country_list_href))
    appendToChecked(country_list_href)
    checked.append(country_list_href)

print("Done")

Pages:  []
Appending  []
Pages:  []
Appending  []




Pages:  [{'BankName': "BANQUE DE L'AGRICULTURE ET DE DEVELOPPEMENT RURAL", 'Branch': '-', 'City': 'ALGER CENTRE', 'SwiftCode': 'BADRDZALXXX'}, {'BankName': 'BANQUE DE DEVELOPPEMENT LOCAL', 'Branch': '-', 'City': 'ALGIERS', 'SwiftCode': 'BDLODZALXXX'}, {'BankName': "CREDIT POPULAIRE D'ALGERIE", 'Branch': '-', 'City': 'ALGIERS', 'SwiftCode': 'CPALDZALXXX'}, {'BankName': 'EPIC ALGERIE POSTE', 'Branch': '-', 'City': 'ALGERIE', 'SwiftCode': 'EPALDZALXXX'}, {'BankName': 'BNP PARIBAS EL DJAZAIR', 'Branch': '-', 'City': 'ALGIERS', 'SwiftCode': 'BNPADZALXXX'}, {'BankName': 'SOCIETE GENERALE ALGERIE', 'Branch': '-', 'City': 'ALGIERS', 'SwiftCode': 'SOGEDZALXXX'}, {'BankName': "BANQUE NATIONALE D'ALGERIE", 'Branch': '-', 'City': 'OUED-SMAR, ALGER', 'SwiftCode': 'BNALDZALXXX'}, {'BankName': 'GULF BANK ALGERIE', 'Branch': '-', 'City': 'ALGIERS, EL BIAR', 'SwiftCode': 'AGUBDZALXXX'}, {'BankName': 'NATIXIS ALGERIE', 'Branch': '-', 'City': 'ALGIERS

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="tableID"]"}
  (Session info: chrome=127.0.6533.89); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF78CDA9632+30946]
	(No symbol) [0x00007FF78CD5E3C9]
	(No symbol) [0x00007FF78CC56FDA]
	(No symbol) [0x00007FF78CCA822C]
	(No symbol) [0x00007FF78CCA850C]
	(No symbol) [0x00007FF78CCEDCB7]
	(No symbol) [0x00007FF78CCCCAAF]
	(No symbol) [0x00007FF78CCEB041]
	(No symbol) [0x00007FF78CCCC813]
	(No symbol) [0x00007FF78CC9A6E5]
	(No symbol) [0x00007FF78CC9B021]
	GetHandleVerifier [0x00007FF78CEDF83D+1301229]
	GetHandleVerifier [0x00007FF78CEEBDB7+1351783]
	GetHandleVerifier [0x00007FF78CEE2A03+1313971]
	GetHandleVerifier [0x00007FF78CDDDD06+245686]
	(No symbol) [0x00007FF78CD6758F]
	(No symbol) [0x00007FF78CD63804]
	(No symbol) [0x00007FF78CD6